In [ ]:
# 使用*和**在函数声明时，用于定位可选参数和关键字参数

# 在调用函数时，使用**来自动拆包可迭代对象传递给函数

def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""   # __doc__属性
    if cls is not None:
        print('-->get cls like %s' % cls)
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

print(tag.__doc__)
print(dir(tag))
print(tag('br'))   # 单个参数，传递给name
print(tag('p', 'hello')) # 第二个参数，传递给*content
print(tag('p', 'hello', id=33)) # 第三个关键字参数，传递给**attrs
print(tag('p', 'hello', 'world', cls='sidebar')) #第三个参数，传递给cls关键字参数
print(tag(content='testing', name="img")) # 第一个参数也可以使用关键字参数形式name

my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
print(tag(**my_tag)) # 自动拆包，除了name，cls被分配给具名参数上，其他都被传递给**attrs

In [ ]:
# 可调用对象，实现__call__方法

import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)  # <1>
        random.shuffle(self._items)  # <2>

    def pick(self):  # <3>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')  # <4>

    def __call__(self):  # <5>
        return self.pick()

bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

In [ ]:
#提取函数信息

def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

print(clip('banana split', 6))
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames) # 获得函数参数（含函数局部变量）
print(clip.__code__.co_argcount) # 获得函数参数个数
print('-----------------------')

from inspect import signature
sig = signature(clip)
print(sig, str(sig))
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)


# 利用bind方法验证参数
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Synset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
for name, value in bound_args.arguments.items():
    print(name, '=', value)

del my_tag['name']
bound_args = sig.bind(**my_tag)

In [ ]:
# 有注解的函数
# 参数之前后有空格，用:增加注释表达式，如果有默认值，注解放在参数名和=之间。
# 函数返回值的注解，在声明末尾于:之间，用->和一个表达式来完成。
# 注解不会做任何处理，只是存储在函数的__annotations__属性中。

def clip(text:str, max_len:'int > 0'=80) -> str:  # <1>
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

sig = signature(clip)
print(sig.return_annotation)
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

In [ ]:
# operator模块代替lambda表达式
# 利用reduce函数不使用递归来实现阶乘
from functools import reduce
from operator import mul

def fact1(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

def fact2(n):
    return reduce(mul, range(1, n+1))

print(fact1(5), fact2(5))

In [ ]:
# 使用operator 模块的itemgetter（使用[]运算符，支持序列、映射以及任何实现__getitem__方法的类）
from operator import itemgetter, attrgetter
from collections import namedtuple

metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

# 利用itemgetter排序一个元组列表
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)
print('---------------')
    
# 多个参数传递给itemgetter，返回提取的值构成的元组
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))
print('---------------')
# attrgetter与itemgetter类似，提取对象的属性
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) 
             for name, cc, pop, (lat, long) in metro_data]
print(metro_areas[0])
print(metro_areas[0].coord.lat)

name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

In [ ]:
# methodcaller 给函数或者方法其别名或者特化
from operator import methodcaller
upcase = methodcaller('upper')
print(upcase('new world!'))

hiph=methodcaller('replace', ' ', '-')
print(hiph('take it down'))

In [ ]:
# functools.partial 偏特化

from operator import mul
from functools import partial

triple = partial(mul, 3)
print(list(map(triple, range(1, 10))))

pic = partial(tag, 'img', cls='pic-frame')
print(pic(src='wupus.jpeg'))
print(pic) # partial返回的functools.partial对象
print(pic.func, '===', tag) # 访问原函数和固定参数的属性